In [1]:
pip install python-dotenv 'crewai[tools]' 'crewai[google-genai]' langchain-google-genai litellm -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [3]:
import os

api_key = os.getenv("GEMINI_API_KEY")

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_openai import ChatOpenAI
from crewai import Agent, Process, Crew, Task
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [26]:
from crewai import LLM


llm = LLM(model="gemini/gemini-2.0-flash", api_key=api_key, temperature=0.5)
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

In [6]:
# interviewer = input("Enter the name of the interviewer: ")
interviewer = "Matthias Glöel"

# Matthias Glöel
company = "Beat81"
# company = input("Enter the name of the company: ")|
#
# job_position = input("Enter the job position: ")
job_position = "Data Analytics Working Student"
# Data Analytics Working Student
job_description = "About the Role As a Working Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g. mathematics, computer science, data science, business informatics etc.)  Good knowledge of Google Sheets  Good Knowledge of SQL  Good Knowledge of Python  Knowledge of common data visualization tools (e.g. Tableau, Lookerstudio etc.)  Knowledge of dbt, GCP, BigQuery is a plus  Fluent in English  OUR OFFER  WALK THE TALK: Unlimited BEAT81 workouts for you and one family member/friend: we guarantee that you will get fit working here :)  YOUR DEVICE: Everyone has their own preferences when it comes to technology - whether it’s Mac or Windows - you choose your laptop. Additionally, we provide Bose noise-canceling headphones  HYBRID WORK: We have a beautiful office in the heart of Berlin (Weinmeisterstraße) with free drinks, healthy snacks and fruits. We like to spend time there together, but you are also welcome to work from home  FLEXIBLE WORKING HOURS: Early bird or late riser? Shape your workday to align with your personal commitments and preferences in coordination with your team and the business needs  TEAM RITUALS: Shared experiences build a strong team culture. That’s why we have our usuals such as regular all hands meetings, Aperol Thursdays and joint workouts - there is something for everyone  TEAM EVENTS: Our summer and Christmas parties have become as legendary as our annual off-sites and the perfect occasion to spend quality time with team members"
# job_description = input("Enter the job description: ")
# About the Role As a Working Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g. mathematics, computer science, data science, business informatics etc.)  Good knowledge of Google Sheets  Good Knowledge of SQL  Good Knowledge of Python  Knowledge of common data visualization tools (e.g. Tableau, Lookerstudio etc.)  Knowledge of dbt, GCP, BigQuery is a plus  Fluent in English  OUR OFFER  WALK THE TALK: Unlimited BEAT81 workouts for you and one family member/friend: we guarantee that you will get fit working here :)  YOUR DEVICE: Everyone has their own preferences when it comes to technology - whether it’s Mac or Windows - you choose your laptop. Additionally, we provide Bose noise-canceling headphones  HYBRID WORK: We have a beautiful office in the heart of Berlin (Weinmeisterstraße) with free drinks, healthy snacks and fruits. We like to spend time there together, but you are also welcome to work from home  FLEXIBLE WORKING HOURS: Early bird or late riser? Shape your workday to align with your personal commitments and preferences in coordination with your team and the business needs  TEAM RITUALS: Shared experiences build a strong team culture. That’s why we have our usuals such as regular all hands meetings, Aperol Thursdays and joint workouts - there is something for everyone  TEAM EVENTS: Our summer and Christmas parties have become as legendary as our annual off-sites and the perfect occasion to spend quality time with team members

# Research Agent

In [27]:
backstory = """Como um Agente de pesquisa, sua missão é descobrir detalhes e informações relevantes.
                O objetivo final é cumprir expectativas da atividade"""
research_agent = Agent(
    role="Agente de pesquisa",
    goal="Conduzir pesquisa aprofundada",
    backstory=backstory,
    llm=llm,
    tools=[ScrapeWebsiteTool(), SerperDevTool()],
)

In [28]:
research_company_task = Task(
    description=f"Pesquise sobre {company}, a posição deles no setor e as trends recentes da industria. Foque em noticias recentes e desenvolvimentos nos ultimos 6 meses",
    expected_output=f"Um report compreensivo em Markdown com bullet points sobre {company}",
    agent=research_agent,
)

In [29]:
research_person_task = Task(
    description=f"Pesquisar sobre {interviewer} com foco na sua carreira profissional",
    expected_output=f"Um report compreensivo em Markdown com bullet points sobre {interviewer}",
    agent=research_agent,
)

# AI Coach

In [30]:
coach_agent = Agent(
    role="Coach de entrevista",
    goal=f"Eu ajudo o usuário a se preparar para a entrevista de emprego para a posição {job_position} avaliando a relevância da resposta.",
    backstory=f"Você é um expert em entrevista de empregos em empresas {company}",
    llm=llm,
)

In [31]:
description = f"Defina uma lista de questões baseada na empresa {company}, descrição de vaga {job_description} e entrevistador {interviewer}"

define_questions_task = Task(
    description=description,
    expected_output="""Uma lista de 20 perguntas no formato Markdown que cubra: Fit cultural e de time, Fit para vaga, Background e jeito de trabalhar, Mentalidade de crescimento""",
    agent=coach_agent,
    context=[research_company_task, research_person_task],
)

In [32]:
# coaching_task = Task(
#     description="Dê feedback para o usuário sobre sua resposta",
#     expected_output="Dê feedback para o usuário sobre sua resposta com bullet points no que foi bom, ruim, e no que pode ser levado para o proximo nível",
#     agent=coach_agent,
#     context=[interview_prep_task],
# )

# AI Crew

In [34]:
crew = Crew(
    agents=[research_agent, coach_agent],
    tasks=[research_person_task, research_company_task, define_questions_task],
    verbose=True,
    process=Process.sequential,
)
result = crew.kickoff(
    {"topic": "Write a list of question to prepare for the interview"}
)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 94660bc0-f7bd-4f4f-9a34-52b75a59c63b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/jonee/Work/crewai-interview/venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Agente de pesquisa                                                                                      │
│                                                                                                                 │
│  Task: Pesquisar sobre Matthias Glöel com foco na sua carreira profissional                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ERROR:root:Google Gemini API error: 429 - You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
Please retry in 43.713157492s.


An unknown error occurred. Please check the details below.
Error details: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash\nPlease retry in 43.713157492s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'ht

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: da46611b-a82e-4793-bf80-791b787a53f4                                                                     │
│  Agent: Agente de pesquisa                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 94660bc0-f7bd-4f4f-9a34-52b75a59c63b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: Google Gemini API error: 429 - You exceeded your current quota, please check your plan and billing      │
│  details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To    │
│  monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit.                                      │
│  * Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0,  │
│  model: gemini-2.0-flash                                                                                        │
│  * Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0,  │
│  model: gemini-2.0-flash                                                                                        │
│  * Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count,   │
│  limit: 0, model: gemini-2.0-flash                                                                              │
│  Please retry in 43.713157492s.                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash\nPlease retry in 43.713157492s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type'

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
display(Markdown(result.tasks_output[2].raw))

NameError: name 'result' is not defined

In [ ]:
# export company report
file_name = f"{company}_report.txt"

with open(file_name, "w") as file:
    file.write(result.tasks_output[0].raw)

# export interviewer questions
file_name = f"{interviewer}_report.txt"

with open(file_name, "w") as file:
    file.write(result.tasks_output[1].raw)

# export interview questions
file_name = f"{job_position}_questions.txt"

with open(file_name, "w") as file:
    file.write(result.tasks_output[2].raw)


# Interview Prep

In [ ]:
with open(f"{job_position}_questions.txt", "r") as file:
    questions = file.read()

# Interview agent

In [ ]:
interviewer_agent = Agent(
    role="Agente Entrevistador",
    goal=f"Faça uma pergunta da lista de perguntas {questions} para o usuário",
    backstory="Você é um entrevistador de uma empresa de tecnologia",
    llm=llm,
)


In [ ]:
interview_prep_task = Task(
    description=f"Entreviste o usuário para a vaga {job_description} com as seguintes descrições: {job_description}",
    expected_output=f"""
Pergunte apenas uma pergunta para o usuário que seja relevante a vaga {job_position}
""",
    agent=interviewer_agent,
    human_input=True,
)

# Feedback Task

In [ ]:
feedback_task = Task(
    description="De feedback para o usuário sobre suas respostas",
    expected_output="Resposta em Markdown do que foi bom, do que pode ser melhorado e como levar para o proximo nível",
    agent=coach_agent,
    context=[interview_prep_task],
    human_input=True,
)

# Interview Prep Crew

In [ ]:
crew = Crew(
    agents=[interviewer_agent, coach_agent],
    tasks=[interview_prep_task, feedback_task],
    verbose=True,
    process=Process.sequential,
)
result = crew.kickoff({"topico": f"Me faça perguntas de {questions}"})
